In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras_metrics import binary_average_recall, binary_recall, binary_precision
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../data/nn_train_data_low_trust.csv", sep="|")
data.describe()

,Unnamed: 0,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedLineItems,lsvc_predict,lsvc_proba,xgb_predict,xgb_proba,fraud
count,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000,679.000000
mean,954.069219,1.511046,911.656848,48.523019,5.543446,4.826215,2.530191,0.047390,0.158916,0.730383,15.359352,0.156112,0.965106,0.148748,0.965443,0.153166
std,554.991652,0.500246,535.079567,29.287414,3.426099,3.186784,1.683308,0.130887,0.727461,1.243906,8.699703,0.363229,0.092778,0.356102,0.085306,0.360413
min,5.000000,1.000000,2.000000,0.070000,0.000000,0.000000,0.000000,0.000551,0.000044,0.000000,1.000000,0.000000,0.500000,0.000000,0.504000,0.000000
25%,452.500000,1.000000,452.500000,23.920000,3.000000,2.000000,1.000000,0.008653,0.025134,0.166667,8.000000,0.000000,0.991500,0.000000,0.986000,0.000000
50%,953.000000,2.000000,900.000000,45.740000,5.000000,5.000000,3.000000,0.016875,0.052280,0.375000,15.000000,0.000000,1.000000,0.000000,0.999000,0.000000
75%,1425.500000,2.000000,1373.000000,74.940000,9.000000,8.000000,4.000000,0.033464,0.113151,0.759615,23.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,1877.000000,2.000000,1830.000000,99.820000,11.000000,10.000000,5.000000,1.611111,17.035000,10.000000,30.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
X = data[["lsvc_predict", "lsvc_proba", "xgb_predict", "xgb_proba"]].values
y = data.fraud.values

In [4]:
model = Sequential()
model.add(Dense(15, input_shape=(4,),activation='relu'))
model.add(Dense(1,activation='sigmoid'))


Instructions for updating:
Colocations handled automatically by placer.


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 15)                75        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 16        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy',binary_precision(), binary_recall(), binary_average_recall()])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)


In [8]:
from keras.callbacks import TensorBoard, EarlyStopping
tensorboard = TensorBoard(log_dir="../logs")
early_stopping = EarlyStopping(monitor='val_precision',min_delta=1e-10, mode=max)

/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/keras/callbacks.py:511: RuntimeWarning: EarlyStopping mode <built-in function max> is unknown, fallback to auto mode.
  RuntimeWarning)


In [12]:
model.fit(X_train, y_train, batch_size=len(X), epochs=1000,verbose=0, validation_data=(X_test,y_test), callbacks=[tensorboard])

In [13]:
model.evaluate(X_test, y_test)

170/170 [==============================] - 0s 20us/step


[0.12920358742532484,
 0.9647058823529412,
 0.9642857108418367,
 0.8437499973632813,
 0.8437499973632813]

In [14]:
import h5py
model.save_weights("../data/nn_weights_low_trust.hdf5")

with open('../data/model_architecture_low_trust.json', 'w') as f:
    f.write(model.to_json())